In [1]:
import csv
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import numpy as np

In [2]:
def show_comparaison(df,feature,feature_p, date_time_key = 'Datetime') : 
    
    # Comparer RMSE(speed,TL_300108) et le RMSE(predicted,TL_300108)
    df["eq_comp"] = (df[feature_p]-df[feature])**2
    RMSE = np.sqrt(df["eq_comp"].mean())

    rRMSE = RMSE/df[feature].mean()*100
    
    df = df.drop('eq_comp', axis=1)

    print('_____________________________')
    print(f"RMSE: {RMSE :.2f} => rRMSE: {rRMSE :.2f} %" )

    trace1 = go.Scatter(
        x=df[date_time_key], 
        y=df[feature], 
        mode='lines',
        line=dict(color='MidnightBlue'), 
        name=feature
    )

    trace2 = go.Scatter(
        x=df[date_time_key],
        y=df[feature_p], 
        mode='lines',
        line=dict(color='orange'),
        name= feature_p
        )

    # Créer la liste des traces
    data = [trace1, trace2]

    titre = "Comparaison " + feature + " et " + feature_p

    # Créer la configuration de la figure
    layout = go.Layout(
        title=titre,
        xaxis=dict(title='Date et heure'),
        yaxis=dict(title='Vitesse du vent')
    )

    # Créer la figure
    fig = go.Figure(data=data, layout=layout)

    # Afficher la figure
    fig.show()

In [3]:
csv_file = 'data/balnea_data_reg.csv'

df = pd.read_csv(csv_file)

features = ['speed', 'direction']
target = 'TL_300108'

X = df[features]
y = df[target]
train_test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_test_size, shuffle=False)

# Détermination du type de problème (classification ou régression)
if set(y.unique()) == {0, 1}:
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None, predictions=True)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
else:
    clf = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None, predictions=True)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [00:02<00:00, 18.48it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000045 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 1525, number of used features: 2
[LightGBM] [Info] Start training from score 2.074308


In [4]:
models[:1].index.to_list()

['NuSVR']

In [5]:
best_model = models[:1].index.to_list()

# Données de test
df_test = df.loc[:int(df.shape[0] * 0.2)]

# Assurez-vous que la colonne 'time' est au bon format
df_test['time'] = pd.to_datetime(df_test['time'])

# Tri par datetime si nécessaire
df_test_sorted = df_test.sort_values(by='time')

# Ajouter les prédictions au DataFrame de test
df_test_sorted['predictions'] = predictions[best_model]

show_comparaison(df_test_sorted,target,'predictions','time')

_____________________________
RMSE: 2.59 => rRMSE: 128.48 %


In [8]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
NuSVR,0.12,0.13,1.91,0.07
MLPRegressor,0.09,0.09,1.94,0.66
SVR,0.06,0.06,1.98,0.08
PoissonRegressor,0.04,0.04,2.00,0.00
BayesianRidge,0.04,0.04,2.00,0.02
RidgeCV,0.03,0.04,2.00,0.01
Ridge,0.03,0.04,2.00,0.00
LassoLarsIC,0.03,0.04,2.00,0.02
Lars,0.03,0.04,2.00,0.00


In [7]:
predictions[models[:3].index.to_list()]

,NuSVR,MLPRegressor,SVR
0,2.10,2.35,1.71
1,2.09,2.33,1.70
2,2.01,2.31,1.61
3,1.44,1.88,1.01
4,1.00,1.15,0.88
...,...,...,...
377,1.06,1.03,0.76
378,1.03,0.85,0.63
379,1.02,0.84,0.60
380,1.02,0.84,0.60
